# 부록: 영화진흥위원회 OPEN API를 이용하여 박스오피스 데이터 모으기 

여기에서는 영화진흥위원회의 [영화입장권통합전사망 OPEN API](http://www.kobis.or.kr/kobisopenapi/homepg/main/main.do)를 이용하여 일일 박스오피스 데이터를 저장합니다. API란 (Application Programing Interface)를 의미하는데, 정부부처나 회사 등의 다양한 서비스에서 만들어진 이용하도록 도와주는 매개체라고 할 수 있습니다. 영화진흥위원회 OPEN API는 허락된 사용자들에게 일일 박스오피스 데이터를 제공해줍니다. API를 활용해 데이터를 수집하거나, 새로운 서비스를 개발할 수도 있습니다. 

이 실습을 통해 얻는 데이터는 11장에서 영화관객수를 예측하는데 사용될 것입니다. 전체 데이터를 다운받는 데 약 두 세 시간 정도가 걸리기 때문에 실습을 원할히 진행할 수 있도록 실습 이전에 데이터를 수집할 필요가 있습니다. 



#### 필요한 패키지 로딩 

In [1]:
import os # 디렉토리 설정을 위한 패키지
import pandas as pd # 데이터 처리를 위한 패키지 
import requests # API를 통해 데이터를 받기 위한 패키지 
import json # API 결과를 데이터 프레임 형태로 바꾸기 위한 패키지 
import datetime  # 날짜 데이터 처리를 위한 패키지 

#### 디렉토리 설정 

영화 데이터를 다운 받아 저장하기 위한 디렉토리를 설정하여야 합니다. 본인이 데이터를 저장하고 싶은 디렉토리의 이름을 아래와 같이 입력하면 됩니다. 

1. `os.envion["HOME"]`을 통해 루트 디렉토리의 경로를 `homePath` 변수에 저장합니다. 
2. `os.path.join`펑션을 통해 데이터를 저장할 디렉토리의 경로를 설정합니다. 예를들어 본인의 루투가 `C:/WWW` 저장하려는 디렉토리의 경로가 `DATA/Download`라면 `os.path.join(homePath, 'DATA', 'Download')`로 변경하면 됩니다.  

In [2]:
# 실제로 데이터를 저장하려는 디렉토리로 설정 

homePath = os.environ["HOME"]
dataHome = os.path.join(homePath, "Google Drive", "00 Academic Work", "_Data", "_SKKU DATA","kobis") # 실제 디렉토리로 설정

print("루트 디렉토리: {}".format(homePath))
print("다운받은 영화데이터가 저장되는 경로:\n {}".format(dataHome))

루트 디렉토리: /Users/kimminc
다운받은 영화데이터가 저장되는 경로:
 /Users/kimminc/Google Drive/00 Academic Work/_Data/_SKKU DATA/kobis


## 일일 박스오피스 데이터 모으기 

영화진흥위원회 Open API를 활용하여 2010년 1월 1일부터 2019년 12월 31일까지 영화흥행순위 데이터를 수집합니다. 하단의 두 코드를 실행하기 위해서는 영화진응위원회 Open API의 Access Key가 필요합니다. Access Key는 영화진흥위 Open API홈페이지를 통해 발급받을 수 있습니다. 우리가 도어락의 문을 여는 데 비밀번호가 필요한 것처럼, Acess Key는 이 API라는 문을 여는 비밀번호라고 생각하면 이해하기 쉽습니다. 

영화진흥위의 Open API는 하루에 3000회까지 사용가능합니다. 이는 API에 3000번 데이터를 받으면 그 날을 더이상 데이터를 달라고 요청할 수 없을을 의미합니다. 우리가 실습을 위해서 총 2010년부터 2019년까지의 박스오피스 데이터를 수집하려고 하기 때문에, 하나의 API Acess Key를 사용하면 데이터를 모으는 데 이틀의 시간이 필요합니다. 다행스럽게도 특별한 조건없이 영화진흥위의 Open API로부터 두 개의 API Acess Key를 발급받을 수가 있습니다. 


**Open API를 통해 박스 오피스 데이터를 수집하는데 상당한 시간이 걸립니다.** 따라서 원활한 실습을 위해서 실습을 하기 이 전에 데이터를 모와둘 필요가 있습니다.

1. 영화진흥위원회의 OPEN API 홈페이지에서 API키 두 개를 발급받습니다.  
2. 발급받은 API키를 아래의 따옴표 안에 붙여넣습니다. 


In [3]:
# 첫번째 API Key 아래의 따옴표 안에 넣으세요. 
api_key1 = "7bc977870033d715d779db40bcbb35f7" 
# 두번째 API Key 아래의 따옴표 안에 넣으세요. 
api_key2 = "46bec287d82ca1eaba036108ffc30077" 
# API Call 주소 
call_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json" 

#### 2010년 1월 1일부터 2020년 9월 30일까지의 데이터 모으기

일일 박스오피스 데이터를 다운로드 받는 과정은 다음과 같습니다. API에서 일일박스오피스 데이터를 받기 위해 필요한 인자는 `API키와`와 `해당날짜`
입니다. 이 두 인자를 API의 주소에 `key=본인의API키`와 `targetDT=YYYYMMDD` 형태로 요청해야 합니다. 

1. for/loop에서 `pandas`의 `date_range`패키지를 통해 20100101부터 20200930까지 날짜 시리즈를 만들어줍니다. 
2. API에서 날짜는 YYYYMMDD 형태로 요구하기 때문에 for/loop에서 각 요소를 적절한 양식으로 변환하여 줍니다. 
3. `parameters`라는 딕셔너리에 API에 데이터를 요청하기 위해 필요한 `API키와`와 `요청날짜`를 저장해 줍니다. 
4. 이 때 각 API키의 일일한계(3000회)를 고려하여 if/else를 사용해 적절한 시기에 API키를 교체해 줍니다.
5. `reuqests` 패키지의 `get` 펑션을 사용하면 파이썬으로 특정 주소를 요청할 수 있습니다. 크롬이나 인터넷 익스플로러와 같은 인터넷 브라우저의 인터넷 주소창에 주소를 입력하는 것과 같은 것이라고 생각하면 됩니다. 실제 인터넷 브라우저와 다른 점은 결과를 인터넷 브라우저에 우리가 알아 볼 수 있는 형태로 표시해주는 것이 아니라, 실제 작성된 `html`코드를 가져온다는 점입니다. 
6. `requests.get`에서 Open API의 주소와 매번 변경되는 `parameters`를 합쳐 결과값을 받아 `res`라는 변수에 저장합니다.  
7. 영화진흥위원회는 OPEN API를 통해 요청한 날에 대한 박스오피스 결과를 JSON형태로 변환하기 위해서 `json` 패키지의 `loads`펑션을 사용합니다. 이 때 반환된 값을 그대로 투입하는 것이 아니라 `res.text`처럼 텍스트 형태로 입력해야 적절한 결과를 얻을 수 있습니다. 
8. 입력된 JSON파일에서 영화박스오피스 데이터만을 선택하여 `temp_df`라는 임시 데이터 프레임에 저장합니다. 
9. 임시로 저장된 `temp_df`를 2010년 1월 1일부터 수집한 전체 데이터와 합칩니다. 
10. 약 100일간의 데이터를 수집했거나, 마지막 날 (여기에서는 2015년 12월 31일)까지의 데이터를 모두 수집한 경우 데이터를 CSV형태로 저장합니다.  


In [4]:
# 데이터를 저장하기 위한 빈데이터 프레임 생성 
df = pd.DataFrame()
# 진행 사항 파악과 API_Key변경을 위해 데이터 요청을 몇 번했는지 저장하기 위한 변수 설정 
i = 0

In [16]:
# 수집한 데이터를 저장하기 위한 빈 데이터 프레임 만들기 
# 진행 상황을 파악하기 위한 변수

# 20100101 부터 20151231일까지를 시리즈로 만듬 
for target_date in pd.date_range(start='20100101',end='20200930',freq='D').strftime('%Y%m%d'):
    
    
    # API에 요청에 필요한  날짜(targetDt) 및 API_키(key)를 설정. 
    # Api 요청 횟수가 2800보다 작으면 Api_key1을 많으면 API_key2를 사용

    if (i//3000<1) and (i%3000<=2900):
        key=1
        parameters = {'key': api_key1, 'targetDt':target_date}
    else:
        key=2
        parameters = {'key': api_key2, 'targetDt':target_date}

    
    # API에 데이터 요청 
    res = requests.get(call_url, params=parameters)
    
    # 결과값을 JSON형태로 변환  
    temp = json.loads(res.text) 
    
    # 필요한 데이터 필드를 임시 데이터 프레임에 저장 
    temp_df=pd.DataFrame.from_dict(temp['boxOfficeResult']['dailyBoxOfficeList'])
    
    # 해당 날짜를 'date' 행에 저장 
    temp_df['date'] = datetime.datetime.strptime(target_date, "%Y%m%d").strftime('%Y-%m-%d')

    # 전체 데이터 프레임과 임시 데이터 프레임과 합침 
    df = df.append(temp_df)
    
    days_left = datetime.datetime.strptime('20200930', "%Y%m%d")-datetime.datetime.strptime(target_date, "%Y%m%d")
    
    # 데이터가 제대로 수집되는지와 진행상황을 확인하기 위해 매일 박스오피스 1위 영화 출력 
    print('{d}일의 흥행 1위 영화: {movie} | {ds}일이 남았습니다 | 현재 API_Key {api}'.format(d=target_date, movie=temp_df['movieNm'].iloc[0], ds=days_left.days, api=key))
    
    # 진행 과정을 저장 
    i=i+1
    
    # 100일 분량의 데이터를 수집했거나, 마지막 날인 경우 데이터를 저장 
    if (i%100==0 or target_date=="20200930"):
        df.to_csv(os.path.join(dataHome,"kobis_daily_boxoffice_df.csv"), sep="\t") # CSV형태로 저장 
        print('\n{}일까지의 데이터를 저장합니다\n'.format(target_date)) # 저장결과 출력 

20100101일의 흥행순위 1위 영화는 아바타입니다. 3925일의 데이터를 더 수집해야합니다
20100102일의 흥행순위 1위 영화는 아바타입니다. 3924일의 데이터를 더 수집해야합니다
20100103일의 흥행순위 1위 영화는 아바타입니다. 3923일의 데이터를 더 수집해야합니다
20100104일의 흥행순위 1위 영화는 아바타입니다. 3922일의 데이터를 더 수집해야합니다
20100105일의 흥행순위 1위 영화는 아바타입니다. 3921일의 데이터를 더 수집해야합니다
20100106일의 흥행순위 1위 영화는 아바타입니다. 3920일의 데이터를 더 수집해야합니다
20100107일의 흥행순위 1위 영화는 아바타입니다. 3919일의 데이터를 더 수집해야합니다
20100108일의 흥행순위 1위 영화는 아바타입니다. 3918일의 데이터를 더 수집해야합니다
20100109일의 흥행순위 1위 영화는 아바타입니다. 3917일의 데이터를 더 수집해야합니다
20100110일의 흥행순위 1위 영화는 아바타입니다. 3916일의 데이터를 더 수집해야합니다
20100111일의 흥행순위 1위 영화는 아바타입니다. 3915일의 데이터를 더 수집해야합니다
20100112일의 흥행순위 1위 영화는 아바타입니다. 3914일의 데이터를 더 수집해야합니다
20100113일의 흥행순위 1위 영화는 아바타입니다. 3913일의 데이터를 더 수집해야합니다
20100114일의 흥행순위 1위 영화는 아바타입니다. 3912일의 데이터를 더 수집해야합니다
20100115일의 흥행순위 1위 영화는 아바타입니다. 3911일의 데이터를 더 수집해야합니다
20100116일의 흥행순위 1위 영화는 아바타입니다. 3910일의 데이터를 더 수집해야합니다
20100117일의 흥행순위 1위 영화는 아바타입니다. 3909일의 데이터를 더 수집해야합니다
20100118일의 흥행순위 1위 영화는 아바타입니다. 3908일의 데이터를 더 수집해야합니다
20100119일의 흥행순위 1위 영화는 아바타입니다. 3907일의 데이터를 더 수

20100531일의 흥행순위 1위 영화는 페르시아의 왕자: 시간의 모래입니다. 3775일의 데이터를 더 수집해야합니다
20100601일의 흥행순위 1위 영화는 페르시아의 왕자: 시간의 모래입니다. 3774일의 데이터를 더 수집해야합니다
20100602일의 흥행순위 1위 영화는 드래곤 길들이기입니다. 3773일의 데이터를 더 수집해야합니다
20100603일의 흥행순위 1위 영화는 방자전입니다. 3772일의 데이터를 더 수집해야합니다
20100604일의 흥행순위 1위 영화는 방자전입니다. 3771일의 데이터를 더 수집해야합니다
20100605일의 흥행순위 1위 영화는 방자전입니다. 3770일의 데이터를 더 수집해야합니다
20100606일의 흥행순위 1위 영화는 방자전입니다. 3769일의 데이터를 더 수집해야합니다
20100607일의 흥행순위 1위 영화는 방자전입니다. 3768일의 데이터를 더 수집해야합니다
20100608일의 흥행순위 1위 영화는 방자전입니다. 3767일의 데이터를 더 수집해야합니다
20100609일의 흥행순위 1위 영화는 방자전입니다. 3766일의 데이터를 더 수집해야합니다
20100610일의 흥행순위 1위 영화는 방자전입니다. 3765일의 데이터를 더 수집해야합니다
20100611일의 흥행순위 1위 영화는 방자전입니다. 3764일의 데이터를 더 수집해야합니다
20100612일의 흥행순위 1위 영화는 방자전입니다. 3763일의 데이터를 더 수집해야합니다
20100613일의 흥행순위 1위 영화는 방자전입니다. 3762일의 데이터를 더 수집해야합니다
20100614일의 흥행순위 1위 영화는 방자전입니다. 3761일의 데이터를 더 수집해야합니다
20100615일의 흥행순위 1위 영화는 방자전입니다. 3760일의 데이터를 더 수집해야합니다
20100616일의 흥행순위 1위 영화는 포화속으로입니다. 3759일의 데이터를 더 수집해야합니다
20100617일의 흥행순위 1위 영화는 포화속으로입니다. 3758일의 데이터를 더 수집해야합니다
20100618일의 

20101026일의 흥행순위 1위 영화는 심야의 F.M.입니다. 3627일의 데이터를 더 수집해야합니다
20101027일의 흥행순위 1위 영화는 심야의 F.M.입니다. 3626일의 데이터를 더 수집해야합니다
20101028일의 흥행순위 1위 영화는 부당거래입니다. 3625일의 데이터를 더 수집해야합니다
20101029일의 흥행순위 1위 영화는 부당거래입니다. 3624일의 데이터를 더 수집해야합니다
20101030일의 흥행순위 1위 영화는 부당거래입니다. 3623일의 데이터를 더 수집해야합니다
20101031일의 흥행순위 1위 영화는 부당거래입니다. 3622일의 데이터를 더 수집해야합니다
20101101일의 흥행순위 1위 영화는 부당거래입니다. 3621일의 데이터를 더 수집해야합니다
20101102일의 흥행순위 1위 영화는 부당거래입니다. 3620일의 데이터를 더 수집해야합니다
20101103일의 흥행순위 1위 영화는 부당거래입니다. 3619일의 데이터를 더 수집해야합니다
20101104일의 흥행순위 1위 영화는 부당거래입니다. 3618일의 데이터를 더 수집해야합니다
20101105일의 흥행순위 1위 영화는 부당거래입니다. 3617일의 데이터를 더 수집해야합니다
20101106일의 흥행순위 1위 영화는 부당거래입니다. 3616일의 데이터를 더 수집해야합니다
20101107일의 흥행순위 1위 영화는 부당거래입니다. 3615일의 데이터를 더 수집해야합니다
20101108일의 흥행순위 1위 영화는 부당거래입니다. 3614일의 데이터를 더 수집해야합니다
20101109일의 흥행순위 1위 영화는 부당거래입니다. 3613일의 데이터를 더 수집해야합니다
20101110일의 흥행순위 1위 영화는 초능력자입니다. 3612일의 데이터를 더 수집해야합니다
20101111일의 흥행순위 1위 영화는 초능력자입니다. 3611일의 데이터를 더 수집해야합니다
20101112일의 흥행순위 1위 영화는 초능력자입니다. 3610일의 데이터를 더 수집해야합니다
20101113일의 흥행순위 1위 영

20110317일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3485일의 데이터를 더 수집해야합니다
20110318일의 흥행순위 1위 영화는 킹스 스피치입니다. 3484일의 데이터를 더 수집해야합니다
20110319일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3483일의 데이터를 더 수집해야합니다
20110320일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3482일의 데이터를 더 수집해야합니다
20110321일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3481일의 데이터를 더 수집해야합니다
20110322일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3480일의 데이터를 더 수집해야합니다
20110323일의 흥행순위 1위 영화는 킹스 스피치입니다. 3479일의 데이터를 더 수집해야합니다
20110324일의 흥행순위 1위 영화는 킹스 스피치입니다. 3478일의 데이터를 더 수집해야합니다
20110325일의 흥행순위 1위 영화는 킹스 스피치입니다. 3477일의 데이터를 더 수집해야합니다
20110326일의 흥행순위 1위 영화는 킹스 스피치입니다. 3476일의 데이터를 더 수집해야합니다
20110327일의 흥행순위 1위 영화는 월드 인베이젼입니다. 3475일의 데이터를 더 수집해야합니다
20110328일의 흥행순위 1위 영화는 킹스 스피치입니다. 3474일의 데이터를 더 수집해야합니다
20110329일의 흥행순위 1위 영화는 킹스 스피치입니다. 3473일의 데이터를 더 수집해야합니다
20110330일의 흥행순위 1위 영화는 킹스 스피치입니다. 3472일의 데이터를 더 수집해야합니다
20110331일의 흥행순위 1위 영화는 위험한 상견례입니다. 3471일의 데이터를 더 수집해야합니다
20110401일의 흥행순위 1위 영화는 위험한 상견례입니다. 3470일의 데이터를 더 수집해야합니다
20110402일의 흥행순위 1위 영화는 위험한 상견례입니다. 3469일의 데이터를 더 수집해야합니다
20110403일의 흥행순위 1위 영화는 위험한 상견례입니다. 3468

20110808일의 흥행순위 1위 영화는 7광구입니다. 3341일의 데이터를 더 수집해야합니다
20110809일의 흥행순위 1위 영화는 7광구입니다. 3340일의 데이터를 더 수집해야합니다
20110810일의 흥행순위 1위 영화는 최종병기 활입니다. 3339일의 데이터를 더 수집해야합니다
20110811일의 흥행순위 1위 영화는 최종병기 활입니다. 3338일의 데이터를 더 수집해야합니다
20110812일의 흥행순위 1위 영화는 최종병기 활입니다. 3337일의 데이터를 더 수집해야합니다
20110813일의 흥행순위 1위 영화는 최종병기 활입니다. 3336일의 데이터를 더 수집해야합니다
20110814일의 흥행순위 1위 영화는 최종병기 활입니다. 3335일의 데이터를 더 수집해야합니다
20110815일의 흥행순위 1위 영화는 최종병기 활입니다. 3334일의 데이터를 더 수집해야합니다
20110816일의 흥행순위 1위 영화는 최종병기 활입니다. 3333일의 데이터를 더 수집해야합니다
20110817일의 흥행순위 1위 영화는 최종병기 활입니다. 3332일의 데이터를 더 수집해야합니다
20110818일의 흥행순위 1위 영화는 최종병기 활입니다. 3331일의 데이터를 더 수집해야합니다
20110819일의 흥행순위 1위 영화는 최종병기 활입니다. 3330일의 데이터를 더 수집해야합니다
20110820일의 흥행순위 1위 영화는 최종병기 활입니다. 3329일의 데이터를 더 수집해야합니다
20110821일의 흥행순위 1위 영화는 최종병기 활입니다. 3328일의 데이터를 더 수집해야합니다
20110822일의 흥행순위 1위 영화는 최종병기 활입니다. 3327일의 데이터를 더 수집해야합니다
20110823일의 흥행순위 1위 영화는 최종병기 활입니다. 3326일의 데이터를 더 수집해야합니다
20110824일의 흥행순위 1위 영화는 최종병기 활입니다. 3325일의 데이터를 더 수집해야합니다
20110825일의 흥행순위 1위 영화는 최종병기 활입니다. 3324일의 데이터를 더 수집해야합니

...

20190703일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 455일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190704일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 454일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190705일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 453일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190706일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 452일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190707일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 451일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190708일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 450일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190709일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 449일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190710일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 448일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190711일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 447일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190712일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 446일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190713일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 445일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190714일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 444일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20190715일의 흥행순위 1위 영화는 스파이더맨: 파 프롬 홈입니다. 443일의 데이터를 

20191022일의 흥행순위 1위 영화는 말레피센트 2입니다. 344일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191023일의 흥행순위 1위 영화는 82년생 김지영입니다. 343일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191024일의 흥행순위 1위 영화는 82년생 김지영입니다. 342일의 데이터를 더 수집해야합니다
20191024일까지의 데이터를 저장합니다
새로운 API키로 변경합니다.
20191025일의 흥행순위 1위 영화는 82년생 김지영입니다. 341일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191026일의 흥행순위 1위 영화는 82년생 김지영입니다. 340일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191027일의 흥행순위 1위 영화는 82년생 김지영입니다. 339일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191028일의 흥행순위 1위 영화는 82년생 김지영입니다. 338일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191029일의 흥행순위 1위 영화는 82년생 김지영입니다. 337일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191030일의 흥행순위 1위 영화는 터미네이터: 다크 페이트입니다. 336일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191031일의 흥행순위 1위 영화는 82년생 김지영입니다. 335일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191101일의 흥행순위 1위 영화는 터미네이터: 다크 페이트입니다. 334일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191102일의 흥행순위 1위 영화는 터미네이터: 다크 페이트입니다. 333일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20191103일의 흥행순위 1위 영화는 터미네이터: 다크 페이트입니다. 332일의 데이터를 더 수집해야합니다
새로운 API키로 변경

20200213일의 흥행순위 1위 영화는 정직한 후보입니다. 230일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200214일의 흥행순위 1위 영화는 정직한 후보입니다. 229일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200215일의 흥행순위 1위 영화는 정직한 후보입니다. 228일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200216일의 흥행순위 1위 영화는 정직한 후보입니다. 227일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200217일의 흥행순위 1위 영화는 정직한 후보입니다. 226일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200218일의 흥행순위 1위 영화는 정직한 후보입니다. 225일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200219일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 224일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200220일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 223일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200221일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 222일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200222일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 221일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200223일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 220일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200224일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 219일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200225일의 흥행순위 1위 영화는 지푸라기라도 잡고 싶은 짐승들입니다. 218일의 데이터를 더 수집해야합니다
새로운 API키로 변

20200605일의 흥행순위 1위 영화는 침입자입니다. 117일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200606일의 흥행순위 1위 영화는 침입자입니다. 116일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200607일의 흥행순위 1위 영화는 침입자입니다. 115일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200608일의 흥행순위 1위 영화는 침입자입니다. 114일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200609일의 흥행순위 1위 영화는 침입자입니다. 113일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200610일의 흥행순위 1위 영화는 결백입니다. 112일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200611일의 흥행순위 1위 영화는 결백입니다. 111일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200612일의 흥행순위 1위 영화는 결백입니다. 110일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200613일의 흥행순위 1위 영화는 결백입니다. 109일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200614일의 흥행순위 1위 영화는 결백입니다. 108일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200615일의 흥행순위 1위 영화는 결백입니다. 107일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200616일의 흥행순위 1위 영화는 결백입니다. 106일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200617일의 흥행순위 1위 영화는 결백입니다. 105일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200618일의 흥행순위 1위 영화는 사라진 시간입니다. 104일의 데이터를 더 수집해야합니다
새로운 API키로 변경합니다.
20200619일의 흥행순위 1위 영화는 사라진 시간입니다. 103일의

20200930일의 흥행순위 1위 영화는 담보입니다. 0일의 데이터를 더 수집해야합니다
20200930일까지의 데이터를 저장합니다
